# Jour 11 - Olist x Freeglisse scraping case

<br>
<center>
<img src="https://res-2.cloudinary.com/crunchbase-production/image/upload/c_lpad,f_auto,q_auto:eco/h8ccwg5vhjcoqu1qfuwv" style="display:inline-block;" width=190>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img alt="Logo Freeglisse" src="https://freeglisse.com/img/ski-destock-logo-1569515072.jpg" width=250 style="display:inline-block;" >

## Case study: étude de l'offre et du positionnement de Freeglisse



**_Contexte_**

Olist aimerait désormais **augmenter son offre sur le segment du sport**, en incitant des marketplaces d'articles de sport à rejoindre la plateforme. Pour affiner sa stratégie vis-à-vis de ce type de vendeurs, Olist a besoin **d'analyser leur offre et leur positionnement**, pour mieux connaitre le marché sur lequel ils évoluent.

En particulier, les Sales d'Olist s'intéressent à **FreeGlisse**, une plateforme de vente en ligne de matériel de ski d'occasion. Leur ojectif est d'emmagasiner le plus d’informations possibles à partir de ce que FreeGlisse affiche sur son site web.
<br><br>

**_Instructions_**

Olist voudrait se concentrer dans un premier temps sur le segment des **skis**. Les informations à étudier incluent : **le nombre d'annonces sur les skis**, les **prix** pratiqués, la **répartition de l’offre entre les différentes sous-catégories**, eventuellement la **profondeur des stocks** (= le nombre de produits disponibles, ou le nombre de tailles disponibles par produit), et tout ce que vous pensez qui serait **intéressant pour analyser le positionnement marché de FreeGlisse**.

Ces informations seraient d’autant plus intéressantes si elles étaient étudiées **par type de ski** (indiqué dans la Fiche Technique de chaque ski). N'hésitez pas à effectuer les regroupements qui vous semblent pertinents, à faire des analyses quantitatives et qualitatives, en utilisant **pandas** et en faisant de beaux **graphiques**. Pour les graphiques, vous utiliserez **Google Data Studio**.

Côté technique, 2 compétences pourront être mises en oeuvre: le **crawling** entre les différentes pages du site, pour observer l’ensemble des offres de skis d’occasion (ici, pas besoin de selenium - des techniques plus simples seront possibles), et le **scraping des pages individuelles**, avec différentes infos à aller chercher.

Pour finir, l'objectif de ce cas est aussi de tirer de ces analyses des **insights business**, en particulier des **conclusions sur le positionnement commercial de FreeGlisse et les spécificités qu'il pourrait apporter à Olist**.
<br><br>

**_Présentation_**

En groupe, vous devrez **analyser les informations issues du scraping**, puis en fin de projet **présenter en environ 5 minutes** votre analyse à des Product Managers d'Olist (jurys de Databirdies), sous forme de **dashboard Data Studio**. L'essentiel est que la présentation soit **visuelle et intelligible par des stakeholders non-tech**.

## liste de nos idées 

In [1]:
## - catégorie(performant, loisir, junior(mixte - fille), type(polyvalent, all mountain, piste...), )
# - taille de ski 
# - Etat matériel ( A B C) 
# - prix 
# - marque (moyen vente, moyen prix ...)


## importation des package

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from ydata_profiling import ProfileReport
import missingno as msno #msno.matrix(df)
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm


## Obtention des URL de chaque page

In [3]:
def get_all_pages():
    urls_page = []
    page_number = 1
 
    for i in range(7):
        i = f"https://freeglisse.com/fr/12-ski-occasion?page={page_number}"
        page_number += 1
        urls_page.append(i)
    print(urls_page)
get_all_pages()

['https://freeglisse.com/fr/12-ski-occasion?page=1', 'https://freeglisse.com/fr/12-ski-occasion?page=2', 'https://freeglisse.com/fr/12-ski-occasion?page=3', 'https://freeglisse.com/fr/12-ski-occasion?page=4', 'https://freeglisse.com/fr/12-ski-occasion?page=5', 'https://freeglisse.com/fr/12-ski-occasion?page=6', 'https://freeglisse.com/fr/12-ski-occasion?page=7']


## Recuperation du code HTML de la page 1

In [4]:
test = requests.get("https://freeglisse.com/fr/12-ski-occasion?page=1")
test.content


b'<!doctype html>\n<html lang="fr-FR">\n\n  <head>\n    \n      \n  <meta charset="utf-8">\n\n\n  <meta http-equiv="x-ua-compatible" content="ie=edge">\n\n\n\n  <title>Ski occasion - Ski d&#039;occasion - Skis occasions (1)</title>\n  \n    \n  \n  <meta name="description" content="Ski occasion">\n  <meta name="keywords" content="ski occasion">\n        <link rel="canonical" href="https://freeglisse.com/fr/12-ski-occasion">\n    \n          <link rel="alternate" href="https://freeglisse.com/fr/12-ski-occasion?page=1" hreflang="fr-fr">\n          <link rel="alternate" href="https://freeglisse.com/de/12-gebrauchte-ski?page=1" hreflang="de-de">\n          <link rel="alternate" href="https://freeglisse.com/en/12-used-ski?page=1" hreflang="en-en">\n          <link rel="alternate" href="https://freeglisse.com/es/12-esqui-usado?page=1" hreflang="es-es">\n          <link rel="alternate" href="https://freeglisse.com/it/12-sci-usato?page=1" hreflang="it-it">\n      \n  \n  \n    <script type="ap

test.content est un objet bytes (un objet binaire)

In [5]:
type(test.content)

bytes

## Mise en soup (meilleure lecture) du "test.content"

In [6]:
soup = BeautifulSoup(test.content)
soup

<!DOCTYPE html>
<html lang="fr-FR">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<title>Ski occasion - Ski d'occasion - Skis occasions (1)</title>
<meta content="Ski occasion" name="description"/>
<meta content="ski occasion" name="keywords"/>
<link href="https://freeglisse.com/fr/12-ski-occasion" rel="canonical"/>
<link href="https://freeglisse.com/fr/12-ski-occasion?page=1" hreflang="fr-fr" rel="alternate"/>
<link href="https://freeglisse.com/de/12-gebrauchte-ski?page=1" hreflang="de-de" rel="alternate"/>
<link href="https://freeglisse.com/en/12-used-ski?page=1" hreflang="en-en" rel="alternate"/>
<link href="https://freeglisse.com/es/12-esqui-usado?page=1" hreflang="es-es" rel="alternate"/>
<link href="https://freeglisse.com/it/12-sci-usato?page=1" hreflang="it-it" rel="alternate"/>
<script type="application/ld+json">
  {
    "@context": "https://schema.org",
    "@type": "Organization",
    "name" : "Freeglisse.com",
    "url" : "https://free

## Ciblage sur la balise "h2" et "class":"h3 product-title"

In [7]:
link = soup.find('h2', {"class":"h3 product-title"}) 
link

<h2 class="h3 product-title"><a content="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm" href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm">Ski occasion Rossignol Sender 104 Ti 2023  + Fixations</a></h2>

## Ciblage sur la balise "a" et extration du lien "href"

In [8]:
link.a.get('href')

'https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm'

seconde méthode

In [9]:
link.find('a', href = True )['href']

'https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm'

## Mise en place d'une boucle FOR pour extraire tout les url des articles (tout les liens "href")

In [10]:
article_link = []
for link in soup.find_all('h2', {"class":"h3 product-title"}):
    article_link.append(link.a.get('href'))

résultat de la boucle

In [11]:
article_link

['https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm',
 'https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18915-431270-ski-occasion-rossignol-sender-94-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/898-taille_skis-156_cm',
 'https://freeglisse.com/fr/ski-de-fond-occasion-alternatif-norme-sns/18974-434106-ski-de-fond-occasion-rossignol-lts-junior-fixation-sns-profil.html#/892-taille_skis-150_cm/1768-etat_du_materiel-qualite_c',
 'https://freeglisse.com/fr/ski-occasion-adulte-performance/19115-487602-ski-occasion-rossignol-react-6-compact-fixations.html#/3-etat_du_materiel-qualite_a/891-taille_skis-149_cm',
 'https://freeglisse.com/fr/ski-de-fond-occasion-alternatif-norme-sns/15468-165924-ski-de-fond-occasion-toutes-marques-fixation-sns-profil.html#/882-taille_skis-140_cm/1768-etat_du_materiel-qualite_c',
 'https://freegliss

In [12]:
len(article_link)

24

il y a bien 24 articles par pages. La len de "article_link" nous prouve que la boucle a bien fonctionner

## Passage a l'obtention de tout les url (d'article) de chaque page

Reutilisation de morceau de code existant

In [13]:
temporary_link = []
all_article_link = []
page_number = 1
for i in range(7):
    i = f"https://freeglisse.com/fr/12-ski-occasion?page={page_number}"
    test = requests.get(i)
    test.content
    soup = BeautifulSoup(test.content)
    for link in soup.find_all('h2', {"class":"h3 product-title"}):
        link.a.get('href')
        temporary_link.append(link.a.get('href'))
    all_article_link.extend(temporary_link)
    page_number += 1
    temporary_link = []


Résultat de la boucle

In [14]:
all_article_link

['https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm',
 'https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18915-431270-ski-occasion-rossignol-sender-94-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/898-taille_skis-156_cm',
 'https://freeglisse.com/fr/ski-de-fond-occasion-alternatif-norme-sns/18974-434106-ski-de-fond-occasion-rossignol-lts-junior-fixation-sns-profil.html#/892-taille_skis-150_cm/1768-etat_du_materiel-qualite_c',
 'https://freeglisse.com/fr/ski-occasion-adulte-performance/19115-487602-ski-occasion-rossignol-react-6-compact-fixations.html#/3-etat_du_materiel-qualite_a/891-taille_skis-149_cm',
 'https://freeglisse.com/fr/ski-de-fond-occasion-alternatif-norme-sns/15468-165924-ski-de-fond-occasion-toutes-marques-fixation-sns-profil.html#/882-taille_skis-140_cm/1768-etat_du_materiel-qualite_c',
 'https://freegliss

In [15]:
len(all_article_link)

157

La boucle fonctionne bien il y a bien 158 articles au moment de l'éxecution

## Extraction du détail produit 

In [16]:
first_article = requests.get("https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm")
first_article.content

b'<!doctype html>\n<html lang="fr-FR">\n\n  <head>\n    \n      \n  <meta charset="utf-8">\n\n\n  <meta http-equiv="x-ua-compatible" content="ie=edge">\n\n\n\n  <title>Ski occasion Rossignol Sender 104 Ti 2023  + Fixations</title>\n  \n    \n  \n  <meta name="description" content="De la poudreuse \xc3\xa0 la neige tass\xc3\xa9e, le nouveau Sender 104 Ti est l\xe2\x80\x99arme absolue pour dompter toute la montagne. Sa facilit\xc3\xa9 et sa r\xc3\xa9activit\xc3\xa9 permettent d\xe2\x80\x99aller s">\n  <meta name="keywords" content="">\n        <link rel="canonical" href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html">\n    \n          <link rel="alternate" href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html" hreflang="fr-fr">\n          <link rel="alternate" href="https://freeglisse.com/de/erwachsener-benutzter-ski-freer

## mise en soup

In [17]:
soup_first_article = BeautifulSoup(first_article.content)
soup_first_article

<!DOCTYPE html>
<html lang="fr-FR">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<title>Ski occasion Rossignol Sender 104 Ti 2023  + Fixations</title>
<meta content="De la poudreuse à la neige tassée, le nouveau Sender 104 Ti est l’arme absolue pour dompter toute la montagne. Sa facilité et sa réactivité permettent d’aller s" name="description"/>
<meta content="" name="keywords"/>
<link href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html" rel="canonical"/>
<link href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html" hreflang="fr-fr" rel="alternate"/>
<link href="https://freeglisse.com/de/erwachsener-benutzter-ski-freeride-freestyle/18916-ski-rossignol-sender-104-ti-2023-gelegenheit-befestigungen.html" hreflang="de-de" rel="alternate"/>
<link href="https://freeglisse.com/en/adult-u

## ciblage sur la balise principal

In [18]:
link_first_article = soup_first_article.find('div', {"class":"tab-content"}) 
link_first_article

<div class="tab-content" id="tab-content">
<div class="tab-pane fade in active js-product-tab-active" id="description" role="tabpanel">
<div class="product-description"><p><span class="js-description-disp" style="font-family:Verdana, Arial;font-size:13px;background-color:#ffffff;">De la poudreuse à la neige tassée, le nouveau Sender 104 Ti est l’arme absolue pour dompter toute la montagne. Sa facilité et sa réactivité permettent d’aller sur n’importe quel terrain, sur piste ou hors piste, grâce à double technologie LCT et Air Tip. Notre technologie d’amortissement des vibrations s’associe à un noyau léger en bois de paulownia pour offrir la polyvalence nécessaire pour la glisse décontractée ou l’attaque à outrance. Franchissez les limites et lancez-vous sans hésitation sur tous les terrains.</span><a class="js-description-toggle" href="https://www.skipass.com/guide-matos/ski/2023/rossignol/sender-104-ti.html#" style="color:#00a4ff;font-family:Verdana, Arial;font-size:13px;background-co

## application du premier filtre 

les valeurs si dessous sont les informations du "détail produit"

In [19]:
filter_one = link_first_article.find('section', {"class":"product-features"})
filter_one

<section class="product-features">
<p class="h6">Fiche technique</p>
<dl class="data-sheet">
<dt class="name">Type</dt>
<dd class="value">Freeride</dd>
<dt class="name">Utilisateur</dt>
<dd class="value">Mixte</dd>
<dt class="name">Niveau</dt>
<dd class="value">Performant</dd>
<dt class="name">Coloris</dt>
<dd class="value">Gris</dd>
<dt class="name">Utilisateur - Configurateur</dt>
<dd class="value">Freerideur adulte</dd>
<dt class="name">Economie de CO2 pour la planète (en kg)</dt>
<dd class="value">3.6</dd>
<dt class="name">Type de produit</dt>
<dd class="value">Ski occasion freeride</dd>
</dl>
</section>

Extraction du text du première article

In [20]:
str(filter_one).split('Type')[1].split('>')[2].split('<')[0]
str(filter_one).split('Utilisateur')[1].split('>')[2].split('<')[0]
str(filter_one).split('Coloris')[1].split('>')[2].split('<')[0]

'Gris'

## Extraction du prix de tout les articles

In [21]:
price = requests.get("https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm")
price.content

b'<!doctype html>\n<html lang="fr-FR">\n\n  <head>\n    \n      \n  <meta charset="utf-8">\n\n\n  <meta http-equiv="x-ua-compatible" content="ie=edge">\n\n\n\n  <title>Ski occasion Rossignol Sender 104 Ti 2023  + Fixations</title>\n  \n    \n  \n  <meta name="description" content="De la poudreuse \xc3\xa0 la neige tass\xc3\xa9e, le nouveau Sender 104 Ti est l\xe2\x80\x99arme absolue pour dompter toute la montagne. Sa facilit\xc3\xa9 et sa r\xc3\xa9activit\xc3\xa9 permettent d\xe2\x80\x99aller s">\n  <meta name="keywords" content="">\n        <link rel="canonical" href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html">\n    \n          <link rel="alternate" href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html" hreflang="fr-fr">\n          <link rel="alternate" href="https://freeglisse.com/de/erwachsener-benutzter-ski-freer

In [22]:
soup_price = BeautifulSoup(price.content)
soup_price

<!DOCTYPE html>
<html lang="fr-FR">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<title>Ski occasion Rossignol Sender 104 Ti 2023  + Fixations</title>
<meta content="De la poudreuse à la neige tassée, le nouveau Sender 104 Ti est l’arme absolue pour dompter toute la montagne. Sa facilité et sa réactivité permettent d’aller s" name="description"/>
<meta content="" name="keywords"/>
<link href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html" rel="canonical"/>
<link href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html" hreflang="fr-fr" rel="alternate"/>
<link href="https://freeglisse.com/de/erwachsener-benutzter-ski-freeride-freestyle/18916-ski-rossignol-sender-104-ti-2023-gelegenheit-befestigungen.html" hreflang="de-de" rel="alternate"/>
<link href="https://freeglisse.com/en/adult-u

In [23]:
filter_price = soup_price.find('span', {"class":"current-price-value"}) 
filter_price

<span class="current-price-value" content="335.2">
                                      335,20 €
                      </span>

In [24]:
str(filter_price).split('content')[1].split('\"')[1]

'335.2'

## Extraction de la marque

In [25]:
maker = requests.get("https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-431280-ski-occasion-rossignol-sender-104-ti-2023-fixations.html#/3-etat_du_materiel-qualite_a/906-taille_skis-164_cm")
maker.content

b'<!doctype html>\n<html lang="fr-FR">\n\n  <head>\n    \n      \n  <meta charset="utf-8">\n\n\n  <meta http-equiv="x-ua-compatible" content="ie=edge">\n\n\n\n  <title>Ski occasion Rossignol Sender 104 Ti 2023  + Fixations</title>\n  \n    \n  \n  <meta name="description" content="De la poudreuse \xc3\xa0 la neige tass\xc3\xa9e, le nouveau Sender 104 Ti est l\xe2\x80\x99arme absolue pour dompter toute la montagne. Sa facilit\xc3\xa9 et sa r\xc3\xa9activit\xc3\xa9 permettent d\xe2\x80\x99aller s">\n  <meta name="keywords" content="">\n        <link rel="canonical" href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html">\n    \n          <link rel="alternate" href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html" hreflang="fr-fr">\n          <link rel="alternate" href="https://freeglisse.com/de/erwachsener-benutzter-ski-freer

In [26]:
maker_soup = BeautifulSoup(maker.content)
maker_soup

<!DOCTYPE html>
<html lang="fr-FR">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<title>Ski occasion Rossignol Sender 104 Ti 2023  + Fixations</title>
<meta content="De la poudreuse à la neige tassée, le nouveau Sender 104 Ti est l’arme absolue pour dompter toute la montagne. Sa facilité et sa réactivité permettent d’aller s" name="description"/>
<meta content="" name="keywords"/>
<link href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html" rel="canonical"/>
<link href="https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html" hreflang="fr-fr" rel="alternate"/>
<link href="https://freeglisse.com/de/erwachsener-benutzter-ski-freeride-freestyle/18916-ski-rossignol-sender-104-ti-2023-gelegenheit-befestigungen.html" hreflang="de-de" rel="alternate"/>
<link href="https://freeglisse.com/en/adult-u

In [27]:
filter_maker = maker_soup.find('img',{'class':"img img-thumbnail manufacturer-logo"})
filter_maker

<img alt="Rossignol" class="img img-thumbnail manufacturer-logo" loading="lazy" src="https://freeglisse.com/img/m/135-small_default.jpg"/>

In [28]:
filter_maker.get('alt')

'Rossignol'

## Mise en place du DataFrame de qualiter A

In [29]:
from unidecode import unidecode
def table_qualite_a():
    base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_a?page={}"
    current_page = 1
    df_global_a = pd.DataFrame(columns=['nom', 'prix', 'lien'])
    while True:
        url = base_url.format(current_page)
        response = requests.get(url)
        if response.status_code != 200:
            print("Erreur lors de la requête HTTP")
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
        noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]
        prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]
        product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]
        data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
        df_current = pd.DataFrame(data)
        df_global_a = pd.concat([df_global_a, df_current], ignore_index=True)
        next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
        if not next_page_link:
            break
        current_page += 1
    return df_global_a
df_a = table_qualite_a()
df_a['Qualité'] = 'Qualité A'
df_a

,nom,prix,lien,Qualité
0,Ski junior Rossignol Star Wars occasion + fixa...,"69,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
1,Ski occasion Rossignol Sender 94 Ti 2023 + Fi...,"287,20 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
2,Ski occasion Rossignol Sender 104 Ti 2023 + F...,"335,20 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
3,Ski occasion junior Dynastar Team speed + Fix...,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
4,Ski occasion Dynastar SPEED ZONE 4x4 78 + fixa...,"199,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
5,Ski occasion Dynastar SPEED ZONE 4x4 82 + fixa...,"249,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
6,Ski occasion Atomic Cloud Q9 + fixations,"299,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-a...,Qualité A
7,Ski occasion junior Tecnopro Team Pulse + fix...,"55,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
8,Ski occasion Volkl RTM 7.4 + fixations,"109,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
9,Ski occasion Rossignol Nova 8 CA 2022 + Fixat...,"209,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-p...,Qualité A


## Mise en place du DataFrame de qualiter B

In [30]:
from unidecode import unidecode
def table_qualite_b():
    base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_b?page={}"
    current_page = 1
    df_global_b = pd.DataFrame(columns=['nom', 'prix', 'lien'])
    while True:
        url = base_url.format(current_page)
        response = requests.get(url)
        if response.status_code != 200:
            print("Erreur lors de la requête HTTP")
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
        noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]
        prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]
        product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]
        data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
        df_current = pd.DataFrame(data)
        df_global_b = pd.concat([df_global_b, df_current], ignore_index=True)
        next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
        if not next_page_link:
            break
        current_page += 1
    return df_global_b
df_b = table_qualite_b()
df_b['Qualité'] = 'Qualité B'
df_b

,nom,prix,lien,Qualité
0,Ski occasion Rossignol Nova 6 + fixations,"139,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-l...,Qualité B
1,Ski occasion junior toutes marques à 19 € + Fi...,"19,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité B
2,Ski occasion K2 Thrilluvit 85 + fixations,"119,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-a...,Qualité B
3,Ski occasion Armada ARV 84 + fixations,"189,00 EUR",https://freeglisse.com/fr/ski-occasion-freesty...,Qualité B
4,Ski occasion Dynastar SPEED ZONE 4x4 78 + fixa...,"159,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité B
...,...,...,...,...
57,Ski occasion K2 Pinnacle 95 + fixation,"149,00 EUR",https://freeglisse.com/fr/ski-occasion-freerid...,Qualité B
58,Ski occasion Volant Pulse White + fixations,"119,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité B
59,Ski occasion Dynastar SPEED ZONE 10 Ti + fixat...,"169,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité B
60,Ski occasion junior Fischer RC4 worldcup GS + ...,"55,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité B


## Mise en place du DataFrame de qualiter C

In [31]:
from unidecode import unidecode
def table_qualite_c():
    base_url = "https://freeglisse.com/fr/12-ski-occasion/s-1/etat_du_materiel-qualite_c?page={}"
    current_page = 1
    df_global_c = pd.DataFrame(columns=['nom', 'prix', 'lien'])
    while True:
        url = base_url.format(current_page)
        response = requests.get(url)
        if response.status_code != 200:
            print("Erreur lors de la requête HTTP")
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        balises_h2 = soup.find_all("h2", {"class": "h3 product-title"})
        noms_produits = [balise_h2.a.text.strip() for balise_h2 in balises_h2]
        prices = [unidecode(span.get_text().strip()) for span in soup.find_all("span", {"class": "price"})]
        product_links = [h2_tag.a.get("href") for h2_tag in balises_h2]
        data = {'nom': noms_produits, 'prix': prices, 'lien': product_links}
        df_current = pd.DataFrame(data)
        df_global_c = pd.concat([df_global_c, df_current], ignore_index=True)
        next_page_link = soup.find("a", {"rel": "next", "class": "next js-search-link"})
        if not next_page_link:
            break
        current_page += 1
    return df_global_c
df_c = table_qualite_c()
df_c['Qualité'] = 'Qualité C'
df_c

,nom,prix,lien,Qualité
0,Ski de fond occasion Toutes marques + fixatio...,"9,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C
1,Ski occasion Rossignol Nova 6 + fixations,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-femme-l...,Qualité C
2,Ski de fond occasion Rossignol LTS Junior + fi...,"19,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C
3,Ski de fond occasion Toutes marques + fixation...,"15,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C
4,Ski de fond occasion Rossignol Progress + fixa...,"19,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C
5,Ski occasion junior Salomon S Race Jr + fixations,"39,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité C
6,Ski de fond occasion Toutes marques + fixation...,"19,00 EUR",https://freeglisse.com/fr/ski-de-fond-occasion...,Qualité C
7,Ski occasion junior Head Supershape + fixations,"45,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité C
8,Ski occasion Elan Sling Shot + fixations,"39,00 EUR",https://freeglisse.com/fr/ski-occasion-freesty...,Qualité C
9,Ski occasion Volkl Revolt 95 + fixation,"69,00 EUR",https://freeglisse.com/fr/ski-occasion-freesty...,Qualité C


## Merge des 3 DataFrame de qualiter

In [32]:
df_all_quality = pd.concat([df_a, df_b, df_c])
df_all_quality

,nom,prix,lien,Qualité
0,Ski junior Rossignol Star Wars occasion + fixa...,"69,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
1,Ski occasion Rossignol Sender 94 Ti 2023 + Fi...,"287,20 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
2,Ski occasion Rossignol Sender 104 Ti 2023 + F...,"335,20 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
3,Ski occasion junior Dynastar Team speed + Fix...,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A
4,Ski occasion Dynastar SPEED ZONE 4x4 78 + fixa...,"199,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A
...,...,...,...,...
46,Ski occasion Nordica Navigator 85 Ca R + fixat...,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C
47,Ski occasion junior Blizzard Racing GS + fixat...,"29,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité C
48,Ski occasion junior Volkl Racetiger GS + fixat...,"29,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité C
49,Ski occasion Salomon X Wing 8 + fixations,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C


## Mise en place du DataFrame

In [33]:
def detail_produit():
    type_list = []
    price_list = []
    maker_list = []
    utilisateur_list = []
    niveau_list = []
    coloris_list = []
    utilisateur_config_list = []
    economie_co2_list = []
    type_produit_list = []
    for url in df_all_quality['lien']:
        s = requests.Session()
        r = s.get(url)
        product = BeautifulSoup(r.content)
        soup_price = BeautifulSoup(r.content)
        maker_soup = BeautifulSoup(r.content)
        product_info = product.find('section', {"class":"product-features"})
        filter_price = soup_price.find('span', {"class":"current-price-value"})
        filter_maker = maker_soup.find('img',{'class':"img img-thumbnail manufacturer-logo"})
        try :
            type_list.append(str(product_info).split('Type')[1].split('>')[2].split('<')[0])
        except :
            type_list.append(np.nan)
        try :
            price_list.append(str(filter_price).split('content')[1].split('\"')[1])
        except :
            price_list.append(np.nan)
        try :
            maker_list.append(filter_maker.get('alt'))
        except :
            maker_list.append(np.nan)
        try :
            utilisateur_list.append(str(product_info).split('Utilisateur')[1].split('>')[2].split('<')[0])
        except :
            utilisateur_list.append(np.nan)
        try :
            niveau_list.append(str(product_info).split('Niveau')[1].split('>')[2].split('<')[0])
        except :
            niveau_list.append(np.nan)
        try :
            coloris_list.append(str(product_info).split('Coloris')[1].split('>')[2].split('<')[0])
        except :
            coloris_list.append(np.nan)
        try :
            utilisateur_config_list.append(str(product_info).split('Utilisateur - Configurateur')[1].split('>')[2].split('<')[0])
        except :
            utilisateur_config_list.append(np.nan)
        try :
            economie_co2_list.append(str(product_info).split('Economie de CO2 pour la planète (en kg)')[1].split('>')[2].split('<')[0])
        except :
            economie_co2_list.append(np.nan)
        try :
            type_produit_list.append(str(product_info).split('Type de produit')[1].split('>')[2].split('<')[0])
        except :
            type_produit_list.append(np.nan)
    #return pd.DataFrame({
        #'url' : url_list, 
        #'price' : price_list,
        #'maker' : maker_list,
        #'type' : type_list,
        #'utilisateur' : utilisateur_list,
        #'niveau' : niveau_list,
        #'coloris' : coloris_list,
        #'utilisateur config' : utilisateur_config_list,
        #'economie co2' : economie_co2_list,
        #'type produit' : type_produit_list
    #})
    df_all_quality['marque_produit'] = maker_list
    df_all_quality['type_pratique'] = utilisateur_config_list
    df_all_quality['utilisateur'] = utilisateur_list
    df_all_quality['niveau'] = niveau_list
    df_all_quality['colori'] = coloris_list
    df_all_quality['economie co2'] = economie_co2_list
    df_all_quality['type'] = type_list
    df_all_quality['type produit'] = type_produit_list
    

affichage du DataFrame

In [34]:
detail_produit()

In [35]:
df_all_quality

,nom,prix,lien,Qualité,marque_produit,type_pratique,utilisateur,niveau,colori,economie co2,type,type produit
0,Ski junior Rossignol Star Wars occasion + fixa...,"69,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A,Rossignol,un junior,Junior,Débutant,Bleu,3.6,Piste,Ski occasion junior loisir
1,Ski occasion Rossignol Sender 94 Ti 2023 + Fi...,"287,20 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A,Rossignol,Freerideur adulte,Mixte,Performant,Noir,3.6,Freeride,Ski occasion freeride
2,Ski occasion Rossignol Sender 104 Ti 2023 + F...,"335,20 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A,Rossignol,Freerideur adulte,Mixte,Performant,Gris,3.6,Freeride,Ski occasion freeride
3,Ski occasion junior Dynastar Team speed + Fix...,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité A,Dynastar,un junior,Junior,Loisir sport,Blanc,3.6,Piste,Ski occasion junior loisir
4,Ski occasion Dynastar SPEED ZONE 4x4 78 + fixa...,"199,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité A,Dynastar,NaN,Mixte,Débutant,Gris,3.6,All mountain,Ski occasion adulte all mountain
...,...,...,...,...,...,...,...,...,...,...,...,...
46,Ski occasion Nordica Navigator 85 Ca R + fixat...,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C,Nordica,NaN,Mixte,Loisir,Orange,3.6,All mountain,Ski occasion adulte all mountain
47,Ski occasion junior Blizzard Racing GS + fixat...,"29,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité C,Blizzard,NaN,Junior,Compétition,Orange,3.6,Racing,Ski occasion junior performance
48,Ski occasion junior Volkl Racetiger GS + fixat...,"29,00 EUR",https://freeglisse.com/fr/ski-occasion-junior-...,Qualité C,Volkl,NaN,Junior,Performant,Rouge,3.6,Piste,Ski occasion junior performance
49,Ski occasion Salomon X Wing 8 + fixations,"59,00 EUR",https://freeglisse.com/fr/ski-occasion-adulte-...,Qualité C,Salomon,NaN,Mixte,Loisir sport,Rouge,3.6,All mountain,Ski occasion adulte all mountain


changement des valeurs text en valeurs numerique

In [36]:
df_all_quality['economie co2'].nunique()

1

affichage des valeurs manquantes

## Bonus: compilation des données Freeglisse

Quelques mois plus tard, Olist vous rappelle. Un partenariat a été noué avec FreeGlisse, et les produits d'occasion du site seront désormais vendus sur la plateforme Olist!

En revanche, les équipes d'Olist n'ont pas le temps de gérer à la main le transfert des annonces d'un site à l'autre. Ils ont besoin de vous pour **rassembler toutes les informations sur les skis d’occasion dans un fichier unique (excel ou csv)**, et pour **enregistrer l’ensemble des photos**, afin de pouvoir rapidement créer des fiches produit sur Olist. Ils comptent sur vous pour leur présenter un dossier bien organisé, afin que le transfert vers leur plateforme soit le plus simple possible.

Si vous avez le temps de vous lancer dans ce projet, vous pouvez inclure une courte partie là-dessus (slide ou section de notebook) dans votre **présentation finale**, afin **d'expliquer aux équipes d'Olist comment l'information est organisée**.